# 1. Import + Inputs

In [415]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import category_encoders as ce


df_train = pd.read_csv('train_heart.csv')
df_test = pd.read_csv('test_heart.csv')

# 2. EDA


In [416]:
# Voeg de gemiddelde waarde van cholesterol in de 0 waarde
cholesterol = df_train['Cholesterol']

# Berekent gemiddelde van cholestrerol zonder 0 waarde
cholesterol_mean = int(round(sum((f:=[c for c in cholesterol if c != 0])) / len(f), 0))
cholesterol.replace(0, cholesterol_mean, inplace=True)

df_train = df_train[df_train['Oldpeak'] >= 0]


In [417]:
# Haal je target kolom uit de data
target_name = df_train.columns[-1]

# Haal je feature kolommen uit de data
feature_cols = df_train.columns[:-1]

In [418]:
# Toon de eerste 10 rijen van de data
df_train.head(10)

,id,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,563,55,M,ASY,135,204,1,ST,126,Y,1.1,Flat,1
1,884,67,M,ASY,160,286,0,LVH,108,Y,1.5,Flat,1
3,694,56,M,ATA,120,236,0,Normal,178,N,0.8,Up,0
4,491,75,M,ASY,170,203,1,ST,108,N,0.0,Flat,1
5,868,51,M,NAP,110,175,0,Normal,123,N,0.6,Up,0
6,370,60,M,ASY,135,242,0,Normal,63,Y,0.5,Up,1
7,497,61,M,ASY,146,241,0,Normal,148,Y,3.0,Down,1
8,58,54,M,ASY,150,365,0,ST,134,N,1.0,Up,0
9,781,50,F,ASY,110,254,0,LVH,159,N,0.0,Up,0
10,318,61,M,ASY,130,242,1,Normal,77,N,2.5,Flat,1


# 3. Encoding

In [419]:
# Haal al de kolommen uit de dataset
X = df_train[feature_cols]

# Verander de strings naar integers
ce_ord = ce.OrdinalEncoder(cols = feature_cols)
X_cat = ce_ord.fit_transform(X)

c:\Users\marni\Documents\School\2ITF\AI\VSC\ML\venv\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
c:\Users\marni\Documents\School\2ITF\AI\VSC\ML\venv\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
c:\Users\marni\Documents\School\2ITF\AI\VSC\ML\venv\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning:

# 4. Split


In [420]:
X = X_cat # Features
y = df_train[target_name]  # Labels
 
# Split dataset into training set and test set
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training, 30% test

# 5. Train


In [421]:
# Build the models
random_forest = RandomForestClassifier(n_estimators=100)

# Train the classifiers
random_forest.fit(x_train, y_train)

x_train

,id,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
352,345,7,1,1,10,55,2,2,32,1,33,2
635,626,19,2,1,3,123,2,1,58,2,4,2
77,76,5,1,1,10,56,2,2,52,1,4,2
300,293,35,2,2,3,94,2,3,39,2,4,2
355,348,32,1,1,3,6,1,3,26,2,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
268,262,26,1,1,3,94,2,3,58,2,10,1
32,31,10,2,3,15,24,2,3,27,2,4,2
589,581,25,1,1,1,188,2,3,39,1,8,1
296,289,8,1,2,2,23,2,3,23,2,4,2


In [422]:
rf_pred = random_forest.predict(x_test)

rf_pred_serie = pd.Series(rf_pred, name="HeartDisease")
 
x_test = pd.merge(x_test, rf_pred_serie, right_index = True,left_index = True)

x_test

,id,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
144,140,32,1,1,34,6,2,3,76,1,10,3,1
35,34,5,1,3,1,25,2,3,21,2,10,1,1
182,178,5,2,2,2,111,2,3,30,2,4,2,1
153,149,21,1,2,10,99,2,3,34,1,4,2,1
48,47,13,1,3,21,37,2,2,29,2,15,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,161,20,1,1,37,104,2,2,70,1,7,3,1
85,84,7,2,1,8,18,2,2,55,2,4,2,0
133,130,3,1,3,8,89,1,2,71,1,5,1,1
183,179,28,1,1,3,112,2,2,26,1,9,1,1


# 6. Evaluation


In [423]:
classification_rep_rf = classification_report(y_test, rf_pred)
print("\nClassification Report:\n", classification_rep_rf)


Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.66      0.72        86
           1       0.75      0.86      0.80       104

    accuracy                           0.77       190
   macro avg       0.77      0.76      0.76       190
weighted avg       0.77      0.77      0.77       190



# 7. Run Test


# 8. to .csv -> Kaggle

In [424]:
x_test[['id', 'HeartDisease']].to_csv('tester.csv', index = False, encoding='utf-8')